In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os
import analyze_covid
import airport_data_explore

# Unload stored dataframes from other notebooks within directory
%store -r

/home/aytang/sfuhome/353project/airport_data_explore.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  canada['Date'] = pd.to_datetime(canada['Date'])


In [2]:
# Haversine Caluclation (Distance between two lat and lon coordinates) retrieved from:
# https://stackoverflow.com/questions/27928/calculate-distance-between-two-latitude-longitude-points-haversine-formula/21623206

def haversine(lat1, lon1, lat2, lon2):
    p = np.pi / 180
    a = 0.5 - np.cos((lat2 - lat1) * p) / 2 + np.cos(lat1 * p) * np.cos(lat2 * p) * (1 - np.cos((lon2 - lon1) * p )) / 2
    n = 12742 * np.arcsin(np.sqrt(a))
    return n  

In [4]:
def getProv():
    return caLocations['Province/State'].unique()
getProv()

array(['Ontario', 'British Columbia', 'Alberta', 'Quebec', 'Saskatchewan',
       'Nova Scotia', 'Manitoba', 'New Brunswick',
       'Newfoundland and Labrador', 'Prince Edward Island',
       'Northwest Territories', 'Yukon'], dtype=object)

In [10]:
def closest(prov):
    d = []
    
    t = caLocations[caLocations["Province/State"] == prov]
    i = airportLocationsCA[airportLocationsCA["State"] == prov]
    size = len(t)
    size2 = len(i)

    for a in range(size2):
        for s in range(size):
            airport = {
                    'country': t.iloc[s]["Country/Region"],
                    'state/province': t.iloc[s]["Province/State"],
                    'lat': t.iloc[s]["Latitude"],
                    'long': t.iloc[s]["Longitude"],
                    'airport': i.iloc[a]["AirportName"],
                    'distance(km)': haversine(t.iloc[s]["Latitude"], t.iloc[s]["Longitude"], i.iloc[a]["lat"], i.iloc[a]["long"])
                }
            d.append(airport)
    d = sorted(d, key = lambda x: x['distance(km)'])
    return d

In [11]:
def calcNearestAirport(prov):
    d = []
    for s in prov:
        d.append(closest(prov))
    return d[0]

calcNearestAirport("Ontario")

[{'country': 'Canada',
  'state/province': 'Ontario',
  'lat': 51.2538,
  'long': -85.3232,
  'airport': 'Toronto Pearson',
  'distance(km)': 943.9968089808716},
 {'country': 'Canada',
  'state/province': 'Ontario',
  'lat': 51.2538,
  'long': -85.3232,
  'airport': 'Hamilton International',
  'distance(km)': 986.1091587236414}]

In [13]:
provList = getProv()
listDf = []

for s in provList:
    i = calcNearestAirport(s)
    listDf.append(pd.DataFrame.from_dict(i))
    
main = pd.concat(listDf)
main

,airport,country,distance(km),lat,long,state/province
0,Toronto Pearson,Canada,943.996809,51.2538,-85.3232,Ontario
1,Hamilton International,Canada,986.109159,51.2538,-85.3232,Ontario
0,Vancouver International,Canada,591.327410,53.7267,-127.6476,British Columbia
0,Edmonton International,Canada,209.684551,53.9333,-116.5765,Alberta
1,Calgary International,Canada,357.768563,53.9333,-116.5765,Alberta
0,Montreal Mirabel,Canada,807.902649,52.9399,-73.5491,Quebec
1,Montreal Trudeau,Canada,830.972598,52.9399,-73.5491,Quebec
0,Halifax International,Canada,28.925848,44.6820,-63.7443,Nova Scotia
1,Halifax International,Canada,28.925848,44.6820,-63.7443,Nova Scotia
0,Winnipeg International,Canada,442.782434,53.7609,-98.8139,Manitoba


In [ ]:
# Canada data not very interesting. 
# TODO: Figure out what to do with it.
# Missing territories/provinces. Figure out what to do with that